In [1]:
from scipy.io import loadmat
import numpy as np
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

In [2]:
data = loadmat('DataforFit2.mat')


In [3]:
y=data['TrainProperty02']
X=data['averagePcScores']

In [4]:
clf = BayesianRidge(compute_score=True)
clf.fit(X, y)


C:\Users\yucel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=True, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

In [5]:
def err(x1,x2):
    error=abs(x2-x1)/x2
#   print (error)
    return error
    

In [6]:
clf.coef_

array([  57.00299436, 1387.84138103, -591.56148533,  705.42090817,
        319.77904377, -557.01158797])

In [7]:
y_hat=clf.predict(X).reshape(16,1)

ols = LinearRegression()
ols.fit(X, y)



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
err(ols.predict(X),y).sum()/16

0.016453059256960844

In [9]:
sum(err(y_hat,y)/16)

array([0.02011245])

In [10]:
y.shape

(16, 1)

In [11]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loo.get_n_splits(X)

16

In [12]:
LOOCVerr=[]
LOOCVerrOLS=[]
LOOCVerrGPR=[]


for train_index, test_index in loo.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
   # print(X_train, X_test, y_train, y_test)
    clf = BayesianRidge(compute_score=True)
    clf.fit(X_train, y_train)
    ols = LinearRegression()
    ols.fit(X_train[:,0:3], y_train)
    y_hat=clf.predict(X_test).reshape(1,1)
    y_hat2=ols.predict(X_test[:,0:3]).reshape(1,1)
    LOOCVerr.append(err(y_hat,y_test))
    LOOCVerrOLS.append(err(y_hat2,y_test))
    noise=0.5
    rbf = ConstantKernel(1.0) * RBF(length_scale=5.0)
    gpr = GaussianProcessRegressor(kernel=rbf, alpha=noise**2)
    gpr.fit(X_train, y_train)
    yhat_3, cov_s = gpr.predict(X, return_cov=True)
    LOOCVerrGPR.append(err(yhat_3,y_test))
    

    











C:\Users\yucel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
np.sum(LOOCVerr)/16

0.07706908879451958

In [14]:
np.sum(LOOCVerrOLS)/16

0.034707255175682045

In [15]:
import GPy

In [16]:
rbf = GPy.kern.RBF(input_dim=1, variance=232.886, lengthscale=0.1216)
gpr = GPy.models.GPRegression(X, y, rbf)

# Fix the noise variance to known value 
gpr.Gaussian_noise.variance = noise**2
gpr.Gaussian_noise.variance.fix()

# Run optimization
#gpr.optimize();

# Display optimized parameter values
display(gpr)

 C:\Users\yucel\Anaconda3\lib\site-packages\GPy\core\gp.py:87: UserWarning:Your kernel has a different input dimension 1 then the given X dimension 6. Be very sure this is what you want and you have not forgotten to set the right input dimenion in your kernel


GP_regression.,value,constraints,priors
rbf.variance,232.886,+ve,
rbf.lengthscale,0.1216,+ve,
Gaussian_noise.variance,0.25,+ve fixed,


In [17]:
# Obtain optimized kernel parameters
l = gpr.rbf.lengthscale.values[0]
sigma_f = np.sqrt(gpr.rbf.variance.values[0])

In [18]:
yyyy=gpr.predict(X)

In [19]:
y

array([[641.7575],
       [586.542 ],
       [602.3725],
       [536.0675],
       [511.0605],
       [766.784 ],
       [700.0695],
       [684.925 ],
       [649.955 ],
       [648.257 ],
       [601.227 ],
       [594.326 ],
       [583.0125],
       [574.538 ],
       [862.9575],
       [676.478 ]])

In [20]:
err(yyyy,y)[0,:,0].sum()/16

0.01950512324893208

In [21]:
np.shape(yyyy)

(2, 16, 1)

In [22]:
Testdata = loadmat('Testdatainfo2.mat')
y_test=Testdata['RealVals']
X_test=Testdata['TestdataAverageScores']

In [23]:
y_hatBLR=clf.predict(X_test).reshape(-1,1)

In [24]:
err(y_hatBLR,y_test).sum()/3

0.04644108247631012

In [25]:
y_hatOLS=ols.predict(X_test[:,0:3]).reshape(-1,1)

In [26]:
err(y_hatOLS,y_test).sum()/4

0.033315374012476744

In [27]:
y_hatOLS

array([[678.13988483],
       [643.93226679],
       [832.53095556],
       [601.23623711]])

In [28]:
LOOCVerrOLS

[array([[0.07918776]]),
 array([[0.02280395]]),
 array([[0.08726092]]),
 array([[0.02658711]]),
 array([[0.01139721]]),
 array([[0.0414295]]),
 array([[0.00274586]]),
 array([[0.01302074]]),
 array([[0.07279424]]),
 array([[0.01390753]]),
 array([[0.00140256]]),
 array([[0.0110982]]),
 array([[0.03024183]]),
 array([[0.01509621]]),
 array([[0.0676156]]),
 array([[0.05872686]])]

In [29]:
import openpyxl


In [30]:
xfile = openpyxl.load_workbook('ErrorValues.xlsx')


In [31]:
sheet = xfile.get_sheet_by_name('2pt-LR')
for i in range(1,17):
    c1 = sheet.cell(row = i+1, column = 1) 
    c1.value =(float(LOOCVerrOLS[i-1]))
errors=err(y_hatOLS,y_test)
for i in range(1,5):
    c1 = sheet.cell(row = i+1, column = 7)
    c1.value =(float(errors[i-1]))
xfile.save('ErrorValues.xlsx')

In [32]:
# For BLR
sheet = xfile.get_sheet_by_name('2pt-BLR')
for i in range(1,17):
    c1 = sheet.cell(row = i+1, column = 1) 
    c1.value =(float(LOOCVerr[i-1]))
errors=err(y_hatBLR,y_test)
for i in range(1,5):
    c1 = sheet.cell(row = i+1, column = 7)
    c1.value =(float(errors[i-1]))
xfile.save('ErrorValues.xlsx')